# Sparkify: An Apache Cassandra Keyspace

This notebook creates an Apache Cassandra keyspace Sparkify for the music app, Sparkify.

The data is modeled on the following 3 queries:

1. artist, song title and song's length for sessionId=338, and itemInSession=4
2. artist, song (sorted by itemInSession) and user (firstName and lastName) for userid=10, sessionid=182
3. every user name (firstName and lastName) for the song 'All Hands Against His Own'

## First PRE-PROCESSING CSV Files for Denormalization

#### Import Python packages 

In [19]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [23]:

full_data_rows_list = [] 

for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 

print(len(full_data_rows_list))


15


### Creating a smaller event data csv file called event_datafile_full by selecting only necessary colmuns from the dataset csv that will be used to insert data into the Apache Cassandra tables

In [24]:
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [25]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


## The generated CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory. 
The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [26]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [27]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [29]:

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Query 1:

### Question-1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Answer 1:

The Question 1 expects Name of the artist, title of the song and length of the track based on sessionId and itemInSession.
As we are working with a NoSQL database, we need to think about the query first which will be used to fetch the data based on which we will create the Table required.

1) The expected output is : "Name of the artist, title of the song and length of the track"
2) Based on : "sessionId and itemInSession"

From the above two points we know the query to get the data will be a SELECT statement like :

"SELECT Name of the artist, title of the song, length of the track FROM TABLE_NAME WHERE sessionId = value AND itemInSession = value"

In [32]:
query = "CREATE TABLE IF NOT EXISTS songs_listned_by_session "
query = query + "(sessionId int, itemInSession int, artist text, song_title text, length float,   PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [33]:

file = 'event_datafile_new.csv'
before
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO songs_listned_by_session (sessionId, itemInSession, artist, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [40]:
query = "select artist,song_title, length from songs_listned_by_session WHERE sessionId=139 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length )

Tamba Trio Quem Quiser Encontrar O Amor 177.18812561035156


## Query 2:
### Question-2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
query2 = "CREATE TABLE IF NOT EXISTS songs_listened_by_user"
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY ((userid, sessionId), itemInSession))"

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [12]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_listened_by_user (userId, sessionId, itemInSession, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query3 = "select artist,song_title, first_name, last_name from songs_listened_by_user WHERE userId=10 and sessionId = 9"
    
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name )

Black Eyed Peas Pump It Sylvie Cruz


## Query 3:
### Question-3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
query5 = "CREATE TABLE IF NOT EXISTS user_listend "
query5 = query5 + "( song_title text, first_name text, last_name text, userId int,  PRIMARY KEY (song_title, userId))"
try:
    session.execute(query5)
except Exception as e:
    print(e)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listend (song_title, first_name, last_name, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [16]:
query3 = "select first_name, last_name from user_listend WHERE song_title = 'Pump It'"
    
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name )

Sylvie Cruz


### Drop the tables before closing out the sessions

In [17]:
query_table1 = "drop table songs_listned_by_session"
try:
    rows = session.execute(query_table1)
except Exception as e:
    print(e)
    
query_table2 = "drop table songs_listened_by_user"
try:
    rows = session.execute(query_table2)
except Exception as e:
    print(e)

query_table3 = "drop table user_listend"
try:
    rows = session.execute(query_table3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()